# Treinamento YOLOv8 - Rock Paper Scissors

Este notebook treina um modelo YOLOv8 para detectar gestos de Pedra, Papel e Tesoura.

**Autor:** Projeto de Visão Computacional  
**Dataset:** Rock Paper Scissors (Roboflow)  
**Modelo:** YOLOv8n (Nano) - otimizado para velocidade e precisão


## 1. Setup e Instalação de Dependências

Instalamos o Ultralytics que contém a implementação do YOLOv8.


In [ ]:
# Instalar Ultralytics YOLOv8
!pip install ultralytics -q

# Verificar instalação
import ultralytics
print(f"Ultralytics versão: {ultralytics.__version__}")

# Verificar GPU disponível
!nvidia-smi


## 2. Importação de Bibliotecas


In [ ]:
from ultralytics import YOLO
import torch
import os
import yaml
from pathlib import Path
import shutil
from IPython.display import Image, display
import matplotlib.pyplot as plt
import numpy as np
from PIL import Image as PILImage

print(f"PyTorch versão: {torch.__version__}")
print(f"CUDA disponível: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")


## 3. Montar Google Drive e Preparar Dataset

**Instruções:**
1. Faça upload da pasta do dataset para seu Google Drive
2. A estrutura deve ser:
   ```
   /content/drive/MyDrive/rock-paper-scissors/
   ├── train/
   ├── valid/
   ├── test/
   └── data.yaml
   ```


In [ ]:
# Montar Google Drive
from google.colab import drive
drive.mount('/content/drive')


In [ ]:
# Definir caminhos
DATASET_PATH = '/content/drive/MyDrive/rock-paper-scissors'
WORK_DIR = '/content/yolov8_rps'

# Criar diretório de trabalho
os.makedirs(WORK_DIR, exist_ok=True)
os.chdir(WORK_DIR)

print(f"Diretório de trabalho: {os.getcwd()}")


## 4. Verificar e Ajustar data.yaml

O arquivo `data.yaml` contém as configurações do dataset.


In [ ]:
# Ler o data.yaml original
with open(f'{DATASET_PATH}/data.yaml', 'r') as f:
    data_config = yaml.safe_load(f)

print("Configuração original do dataset:")
print(data_config)

# Ajustar caminhos para ambiente Colab
data_config['train'] = f'{DATASET_PATH}/train/images'
data_config['val'] = f'{DATASET_PATH}/valid/images'
data_config['test'] = f'{DATASET_PATH}/test/images'

# Salvar nova configuração
new_yaml_path = f'{WORK_DIR}/data.yaml'
with open(new_yaml_path, 'w') as f:
    yaml.dump(data_config, f)

print("\nNova configuração ajustada:")
print(data_config)


## 5. Análise Exploratória do Dataset


In [ ]:
# Contar imagens em cada conjunto
train_images = len(os.listdir(f'{DATASET_PATH}/train/images'))
val_images = len(os.listdir(f'{DATASET_PATH}/valid/images'))
test_images = len(os.listdir(f'{DATASET_PATH}/test/images'))

print(f"Imagens de Treinamento: {train_images}")
print(f"Imagens de Validação: {val_images}")
print(f"Imagens de Teste: {test_images}")
print(f"Total: {train_images + val_images + test_images}")

# Distribuição
print(f"\nDistribuição:")
print(f"Train: {train_images/(train_images+val_images+test_images)*100:.1f}%")
print(f"Val: {val_images/(train_images+val_images+test_images)*100:.1f}%")
print(f"Test: {test_images/(train_images+val_images+test_images)*100:.1f}%")


In [ ]:
# Visualizar algumas imagens do dataset
import random

train_img_dir = f'{DATASET_PATH}/train/images'
sample_images = random.sample(os.listdir(train_img_dir), min(6, len(os.listdir(train_img_dir))))

fig, axes = plt.subplots(2, 3, figsize=(15, 10))
axes = axes.flatten()

for idx, img_name in enumerate(sample_images):
    img_path = os.path.join(train_img_dir, img_name)
    img = PILImage.open(img_path)
    axes[idx].imshow(img)
    axes[idx].set_title(img_name)
    axes[idx].axis('off')

plt.tight_layout()
plt.show()


## 6. Configuração de Hiperparâmetros

### Justificativa das Escolhas:

- **Modelo YOLOv8n**: Versão Nano escolhida por oferecer excelente equilíbrio entre velocidade e precisão, ideal para aplicações em tempo real.
- **Epochs: 100**: Número suficiente para convergência sem overfitting, com early stopping.
- **Batch Size: 16**: Balanceado para GPU do Colab (T4/P100).
- **Image Size: 640**: Tamanho padrão do YOLO, bom para detectar objetos de tamanhos variados.
- **Learning Rate: 0.01**: Taxa de aprendizado inicial do YOLO, com scheduler automático.
- **Patience: 50**: Early stopping após 50 epochs sem melhora no mAP.


In [ ]:
# Hiperparâmetros de treinamento
HYPERPARAMETERS = {
    'model': 'yolov8n.pt',
    'data': new_yaml_path,
    'epochs': 100,
    'batch': 16,
    'imgsz': 640,
    'lr0': 0.01,
    'lrf': 0.01,
    'momentum': 0.937,
    'weight_decay': 0.0005,
    'warmup_epochs': 3.0,
    'patience': 50,
    'device': 0,
    'workers': 8,
    'project': 'runs/detect',
    'name': 'rps_yolov8n',
    'exist_ok': True,
    'pretrained': True,
    'optimizer': 'auto',
    'verbose': True,
    'seed': 42,
}

print("Hiperparâmetros configurados:")
for key, value in HYPERPARAMETERS.items():
    print(f"  {key}: {value}")


## 7. Treinamento do Modelo

⚠️ **Atenção**: Este processo pode levar de 30 minutos a 2 horas dependendo da GPU disponível.


In [ ]:
# Carregar modelo pré-treinado
model = YOLO('yolov8n.pt')

# Treinar modelo
results = model.train(
    data=HYPERPARAMETERS['data'],
    epochs=HYPERPARAMETERS['epochs'],
    batch=HYPERPARAMETERS['batch'],
    imgsz=HYPERPARAMETERS['imgsz'],
    lr0=HYPERPARAMETERS['lr0'],
    lrf=HYPERPARAMETERS['lrf'],
    momentum=HYPERPARAMETERS['momentum'],
    weight_decay=HYPERPARAMETERS['weight_decay'],
    warmup_epochs=HYPERPARAMETERS['warmup_epochs'],
    patience=HYPERPARAMETERS['patience'],
    device=HYPERPARAMETERS['device'],
    workers=HYPERPARAMETERS['workers'],
    project=HYPERPARAMETERS['project'],
    name=HYPERPARAMETERS['name'],
    exist_ok=HYPERPARAMETERS['exist_ok'],
    pretrained=HYPERPARAMETERS['pretrained'],
    optimizer=HYPERPARAMETERS['optimizer'],
    verbose=HYPERPARAMETERS['verbose'],
    seed=HYPERPARAMETERS['seed'],
)

print("\n✅ Treinamento concluído!")


## 8. Visualização dos Resultados de Treinamento


In [ ]:
# Caminho para os resultados
results_path = Path('runs/detect/rps_yolov8n')

# Visualizar curvas de treinamento
results_img = results_path / 'results.png'
if results_img.exists():
    display(Image(filename=str(results_img)))

# Confusion Matrix
confusion_matrix_img = results_path / 'confusion_matrix.png'
if confusion_matrix_img.exists():
    print("Matriz de Confusão:")
    display(Image(filename=str(confusion_matrix_img)))

# F1 Curve e PR Curve
f1_curve_img = results_path / 'F1_curve.png'
if f1_curve_img.exists():
    display(Image(filename=str(f1_curve_img)))

pr_curve_img = results_path / 'PR_curve.png'
if pr_curve_img.exists():
    display(Image(filename=str(pr_curve_img)))


## 9. Validação no Conjunto de Teste


In [ ]:
# Carregar melhor modelo treinado
best_model = YOLO(results_path / 'weights' / 'best.pt')

# Validar no conjunto de teste
metrics = best_model.val(data=new_yaml_path, split='test')

print("\n📊 Métricas no Conjunto de Teste:")
print(f"mAP50: {metrics.box.map50:.4f}")
print(f"mAP50-95: {metrics.box.map:.4f}")
print(f"Precision: {metrics.box.mp:.4f}")
print(f"Recall: {metrics.box.mr:.4f}")


## 10. Inferência em Imagens de Teste


In [ ]:
# Realizar predições em algumas imagens de teste
test_img_dir = f'{DATASET_PATH}/test/images'
sample_test_images = random.sample(os.listdir(test_img_dir), min(6, len(os.listdir(test_img_dir))))

for img_name in sample_test_images:
    img_path = os.path.join(test_img_dir, img_name)
    
    # Fazer predição
    results = best_model.predict(source=img_path, conf=0.25, save=False)
    
    # Visualizar resultado
    result_img = results[0].plot()
    
    plt.figure(figsize=(10, 8))
    plt.imshow(result_img[..., ::-1])
    plt.title(f"Predição: {img_name}")
    plt.axis('off')
    plt.show()
    
    # Mostrar detecções
    if len(results[0].boxes) > 0:
        print(f"\nDetecções em {img_name}:")
        for box in results[0].boxes:
            cls = int(box.cls[0])
            conf = float(box.conf[0])
            class_name = data_config['names'][cls]
            print(f"  - {class_name}: {conf:.2%}")
    print("-" * 50)


## 11. Análise de Performance e Exportação

Vamos medir a velocidade de inferência e exportar o modelo.


In [ ]:
import time

# Testar velocidade de inferência
test_images_list = [os.path.join(test_img_dir, img) for img in os.listdir(test_img_dir)[:50]]

# Warmup
for _ in range(10):
    best_model.predict(test_images_list[0], verbose=False)

# Medir tempo
start_time = time.time()
for img_path in test_images_list:
    results = best_model.predict(img_path, verbose=False)
end_time = time.time()

total_time = end_time - start_time
avg_time = total_time / len(test_images_list)
fps = 1 / avg_time

print("\n⚡ Performance de Inferência:")
print(f"Tempo total para {len(test_images_list)} imagens: {total_time:.2f}s")
print(f"Tempo médio por imagem: {avg_time*1000:.2f}ms")
print(f"FPS estimado: {fps:.1f}")


In [ ]:
# Copiar modelo para o Drive
best_pt_path = results_path / 'weights' / 'best.pt'
drive_model_path = '/content/drive/MyDrive/rock-paper-scissors/best_yolov8n.pt'
shutil.copy(best_pt_path, drive_model_path)
print(f"Modelo .pt salvo no Drive: {drive_model_path}")

# Copiar pasta de resultados completa
drive_results_path = '/content/drive/MyDrive/rock-paper-scissors/results_yolov8n'
if os.path.exists(drive_results_path):
    shutil.rmtree(drive_results_path)
shutil.copytree(results_path, drive_results_path)
print(f"Resultados salvos no Drive: {drive_results_path}")


## 12. Conclusões

### Pontos Fortes do YOLOv8:
- ✅ Alta precisão na detecção de múltiplas mãos simultaneamente
- ✅ Robusto a diferentes ângulos e condições de iluminação
- ✅ Detecta objetos com bounding boxes precisos
- ✅ Pré-treinado em COCO facilita transfer learning
- ✅ Excelente documentação e comunidade ativa

### Limitações:
- ❌ Requer dataset rotulado com bounding boxes
- ❌ Treinamento demora e requer GPU potente
- ❌ Modelos maiores podem ser lentos em tempo real
- ❌ Necessita fine-tuning para cada caso de uso

### Próximos Passos:
1. Usar o modelo treinado no script de inferência em tempo real
2. Comparar com MediaPipe para análise comparativa
3. Otimizar para produção se necessário
